In [ ]:
# %%
"""
pseudocode:

Function([] optimal_values, hashmap(scenario, matrix) X) <- (A, Omega, P, b, X_inital, tasks, teams)
num_tasks <- len(tasks)
num_teams <- len(teams)
num_scenarios <- len(Omega)

for i in range(num_scenarios):
    X_scenario = sdpvar(num_tasks, num_teams)
    constraints = []
    OBJ = sum(elementwise_multiplication(X_scenario, P, A))
    for i in range(num_teams):
        constraints.append(X_scenario[i, :] <= b[i])
        constraints.append(sum(elementwise_multiplication(X_scenario[i, :], A[i, :], P[i,: ]) <= sum(elementwise_multiplication(X_init[i, :], A[i, :], P[i,: ]))

        for j in range(num_tasks):
            constraints.append(X_scenario[:, j] <= 1)
    optiaml_values.append(solve(constraints, OBJ))
        
"""





In [ ]:

# %%
""""
Pseucode solving cycle problem:
current_phase = matrix of size (num_teams, num_tasks)

Function update_phase <- (current_phase):
    extended_graph = extend_graph(current_phase)
    update_phase = find_better_cycle(extended_graph)


"""


In [ ]:
# %%
"""
Pseudocode for extend_graph:
Function extend_graph <- (current_phase, num_tasks, total_capacity)
    num_dummies = total_capicity - num_tasks
    dim_x, dim_y = current_phase.shape()
    dummy = zeros(dim_x, dim_y + num_dummies)
    extended_graph = list_comprehension(current_phase, dummy)
"""


In [ ]:

# %%
"""
Function identify_cycle <- (extended_graph)
    cycle = extendended_graph(extended_graph > 0)
    return cycle
"""




In [ ]:
# %%
"""
Function find_better_cycle <- (extended_graph)
    current_phase_sum = sum(identify_cycle(extended_graph))
    next_graph = extended_graph - extended_graph(extended_graph > 0)
    next_graph = -next_graph
    dim_x, dimx_y = next_graph.shape()
    select_new_cycle = sdpvar(dim_x, dim_y)
    OBJ = sum(elementwise_multiplication(select_new_cycle, next_graph))
    constraints = []
    for i in range(dim_x):
        constraints.append(sum(select_new_cycle[i, :]) == 1)
    for j in range(dim_y):
        constraints.append(sum(select_new_cycle[:, j]) == 1)
    optimise(constraints, OBJ)
    
    if sum(identify_cycle(next_graph)) > current_phase_sum:
        return next_graph
    else:
        return "best cycle found"
"""

Different method:

Solving matching problem but team may only switch once if and only if both teams are equally or better off

In [ ]:

def switch_tasks(group_1, group_2, current_phase, task_1, task_2):
    current_phase[group_1, task_1] = 0
    current_phase[group_1, task_2] = 1
    current_phase[group_2, task_1] = 1
    current_phase[group_2, task_2] = 0
    return current_phase


In [ ]:
#Find group from task 
def find_group(task, current_phase):
    for group in range(current_phase.shape[0]):
        if current_phase[group, task] == 1:
            return group
    return -1

def find_payout(group, current_phase, real_payouts):
    payout = 0
    for task in range(current_phase.shape[1]):
        if current_phase[group, task] == 1:
            payout += real_payouts[group, task]
    return payout


def best_swap(group_i, current_phase, real_payouts):
    num_tasks = current_phase.shape[1]
    best_payout = find_payout(group_i, current_phase, real_payouts)

    best_swap_i = -1
    best_swap_j = -1
    #set group_j to -1 to check if it has been set

    for task_m in range(num_tasks):
        for task_n in range(num_tasks):
            if task_m != task_n:
                # Swap tasks between group_i and group_j

                # Find group_j
                group_j = find_group(task_n, current_phase)
                group_j_prev_payout = find_payout(group_j, current_phase, real_payouts)


                new_phase = current_phase.copy()
                new_phase[group_i, task_n] = 0
                new_phase[group_i, task_m] = 1

                new_phase[group_j, task_m] = 1
                new_phase[group_j, task_n] = 0

                # Calculate the payout for group_i in the new phase
                group_i_payout = 0
                for task in range(new_phase.shape[1]):
                    if new_phase[group_i, task] == 1:
                        group_i_payout += real_payouts[group_i, task]

                # Calculate the payout for group_j in the new phase
                group_j_next_payout = find_payout(group_j, new_phase, real_payouts)

                # Check if the new payout for group_i is better than the previous best payout
                if group_i_payout > best_payout & group_j_next_payout > group_j_prev_payout:
                    best_payout = group_i_payout
                    best_swap_i = task_m
                    best_swap_j = task_n
                    

    return best_swap_i, best_swap_j
#this function is O(n^3) where n is the number of tasks


Example code run

In [ ]:
import numpy as np
""""
We have the current variables current_phase, real_payouts. 
Since this is a greedy algorithm, we will always try to find the best swap for each group. 
But this does not always lead to the absolute best since we iterate from group 0 to group n.
So we will simulate it with 500 iterations, where each iteration is the "current_phase" but with permuted rows.
"""

import numpy as np

def total_payout(current_phase, real_payouts):
    return sum(sum(current_phase * real_payouts))

def start_simulation(current_phase, real_payouts, iterations):
    num_teams = current_phase.shape[0]
    mean_list = []
    for i in range(iterations):
        permutation = np.random.permutation(num_teams)
        random_phase = current_phase.copy()
        random_phase = random_phase[permutation, :]
        for group in range(num_teams):
            best_swap_i, best_swap_j = best_swap(group, random_phase, real_payouts)
            if best_swap_i != -1:
                random_phase = switch_tasks(group, best_swap_i, best_swap_j, random_phase)
        payout = total_payout(random_phase, real_payouts)
        mean_list.append(payout)
        
    return np.mean(mean_list), np.std(mean_list)
        
    


In [ ]:
#in a realistic sense two teams woudld discuss and agree on a swap instead 
#of a group going through every other group to determine best swap


def two_team_best_swap(group_i, group_j, current_phase, real_payouts):
    num_tasks = current_phase.shape[1]
    payout_i = find_payout(group_i, current_phase, real_payouts)
    payout_j = find_payout(group_j, current_phase, real_payouts)
    best_swap_i = -1
    best_swap_j = -1

    for task_m in range(num_tasks):
        for task_n in range(num_tasks):
            if task_m != task_n:
                new_phase = current_phase.copy()
                new_phase[group_i, task_n] = 0
                new_phase[group_i, task_m] = 1

                new_phase[group_j, task_m] = 1
                new_phase[group_j, task_n] = 0

                group_i_payout = find_payout(group_i, new_phase, real_payouts)
                group_j_payout = find_payout(group_j, new_phase, real_payouts)
                if group_i_payout > payout_i & group_j_payout > payout_j:
                    payout_i = group_i_payout
                    payout_j = group_j_payout
                    best_swap_i = task_m
                    best_swap_j = task_n
    return best_swap_i, best_swap_j

In [ ]:
#simulation based approach of teams swapping 
""""
number_of_iterations = 1000
number_of_teams = current_phase.shape[0]

"""


def simulate_two_team_swap(current_phase, real_payouts, iterations):
    num_teams = current_phase.shape[0]
    mean_list = []
    for i in range(iterations):
        permutation = np.random.permutation(num_teams)
        random_phase = current_phase.copy()
        random_phase = random_phase[permutation, :]
        pairs = [(i, i+1) for i in range(1, num_teams+1, 2)]     


        for pair in pairs:
            group_i, group_j = pair
            best_swap_i, best_swap_j = two_team_best_swap(group_i, group_j, random_phase, real_payouts)
            if best_swap_i != -1:
                random_phase = switch_tasks(group_i, group_j, best_swap_i, best_swap_j, random_phase)
        payout = total_payout(random_phase, real_payouts)
        mean_list.append(payout)
        
    return np.mean(mean_list), np.std(mean_list)
 

        


In [7]:
# import numpy as np

# matrix = np.zeros((5, 8), dtype=int)
# for j in range(8):
#     column = np.random.choice(5, size=1)
#     matrix[column, j] = 1

# print(matrix)

# permute = np.random.permutation(5)
# print(permute)
# print(matrix[permute, :])


[[1 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1]
 [0 0 1 0 0 0 1 0]
 [0 1 0 0 1 1 0 0]]
[3 2 1 0 4]
[[0 0 1 0 0 0 1 0]
 [0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0]
 [1 0 0 1 0 0 0 0]
 [0 1 0 0 1 1 0 0]]


Group meetings assignment.

In [6]:
#Get list of distances
def get_distances(team, geocodes):
    distances = dict()
    geocodes.remove(geocodes[team])
    coordinates_base_team = geocodes[team]
    for i in range(len(geocodes)):
        dist = np.linalg.norm(coordinates_base_team - geocodes[i])
        distances[i] = (f"team {i}", dist)
    return distances

In [1]:
#Baseline
import random 

def baseline(team, geocodes):
    distances = get_distances(team, geocodes)
    sorted_distances = sorted(distances.items(), key=lambda x: x[1][1])
    return random.choice(sorted_distances)
    



In [13]:
#maximum sites
def num_tasks(team, current_phase):
    return sum(current_phase[team, :])

def max_sites(current_phase):
    tasks_per_team = [num_tasks(i, current_phase) for i in range(current_phase.shape[0])]
    max_site_team = tasks_per_team.index(max(tasks_per_team))
    min_site_team = tasks_per_team.index(min(tasks_per_team))
    return max_site_team, min_site_team


In [ ]:
#compementing teams
#Similar to two team swap but will only look at k nearest teams
def complementing_team_swap(team, geocodes, current_phase, payouts):
    distances = get_distances(team, geocodes)
    sorted_distances = sorted(distances.items(), key=lambda x: x[1][1])
    k = 5
    nearest_teams = sorted_distances[:k]
    other_team = random.choice(nearest_teams)
    best_swap_i, best_swap_j = two_team_best_swap(team, other_team, current_phase, payouts)
    return best_swap_i, best_swap_j, other_team
